# Outline
- clean text
- tf-idf text for weights
- embeddings
    - word2vec words * tf-idf
    - save to each row of each df

In [1]:
import time
import pickle
import pandas as pd


import gensim
import gensim.corpora as corpora
from gensim.models import TfidfModel
from gensim.sklearn_api import TfIdfTransformer


# import gensim.models.phrases


In [2]:
### load df

DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df_cleaned.pkl'

with open(DATA_PATH + DF_NAME, 'rb') as f:
    df = pickle.load(f)

### load dictionary
articles_dict_filename = 'articles_dict.pkl'

with open(DATA_PATH + articles_dict_filename, 'rb') as f:
    articles_dict = pickle.load(f)

### load corpus
articles_corpus_filename = 'articles_corpus.pkl'

with open(DATA_PATH + articles_corpus_filename, 'rb') as f:
    articles_corpus = pickle.load(f)

In [3]:
df.head()

,created_at,user,id_str,retweeted_status,Hash words,link,entities,Topic Label,urls,no_link,...,day_of_tweet,month_year,week_month_year,user_id,cluster,is_retweet,hashtags,link_shorteners,article_text,article_text_cleaned
0,2020-01-31 23:08:33+00:00,"{'id': 19031057, 'id_str': '19031057', 'name':...",1223382589689356288,NaN,#coronavirus,https://twitter.com/user/status/12233825896893...,"{'hashtags': [{'text': 'coronavirus', 'indices...","(43, 0.999852366207409)",[https://www.kron4.com/news/national/when-will...,False,...,2020-01-31,2020-01,5,19031057,43,False,[coronavirus],[False],"KRON4 by: Aubree Gordon, University of Michig...","[kron, aubree, gordon, university, michigan, c..."
1,2020-01-31 23:11:29+00:00,"{'id': 798925214, 'id_str': '798925214', 'name...",1223383328843280384,{'created_at': 'Fri Jan 31 20:41:20 +0000 2020...,#coronavirus #vaccine:,https://twitter.com/user/status/12233833288432...,"{'hashtags': [{'text': 'Coronavirus', 'indices...","(6, 0.9883827140131066)",[https://www.cnbc.com/2020/01/31/coronavirus-w...,False,...,2020-01-31,2020-01,5,798925214,6,True,"[Coronavirus, vaccine]",[False],U.S. and international health officials are s...,"[international, health, officials, speeding, w..."
2,2020-01-31 23:12:03+00:00,"{'id': 806144538049970176, 'id_str': '80614453...",1223383471999127552,NaN,No hashtags,https://twitter.com/user/status/12233834719991...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999928079220538)",[https://www.businessinsider.com/australia-suc...,False,...,2020-01-31,2020-01,5,806144538049970176,74,False,[None],[False],A leading-edge research firm focused on digit...,"[leading_edge, research, firm_focused, active,..."
3,2020-01-31 23:35:42+00:00,"{'id': 61298849, 'id_str': '61298849', 'name':...",1223389423045206016,{'created_at': 'Tue Jan 28 18:26:16 +0000 2020...,No hashtags,https://twitter.com/user/status/12233894230452...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(36, 0.9999993854913211)",[https://www.greenmedinfo.com/blog/examining-r...,False,...,2020-01-31,2020-01,5,61298849,36,True,[None],[False],Mr. Kennedy is in very safe territory by rep...,"[mr, kennedy, safe, territory, reporting, cdc,..."
4,2020-01-31 23:37:23+00:00,"{'id': 1152822375567654912, 'id_str': '1152822...",1223389844719599616,NaN,No hashtags,https://twitter.com/user/status/12233898447195...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999985797901156)",[https://www.dailymail.co.uk/news/article-7952...,False,...,2020-01-31,2020-01,5,1152822375567654912,74,False,[None],[True],By Kylie Stevens and Stephen Gibbs and Nic Wh...,"[kylie, stevens, stephen, gibbs, nic, white, d..."


In [ ]:
tfidf_model = TfidfModel(corpus = articles_corpus, dictionary = articles_dict)
corpus_tfidf = tfidf_model[articles_corpus]
corpus_tfidf[0]

In [ ]:
### save tfidf model

DATA_PATH = './models/'
tfidf_name = 'tfidf.model'

tfidf_model.save(DATA_PATH + tfidf_name)

## sklearn tfidf

In [5]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer


df['article_text_cleaned_string'] = df['article_text_cleaned'].apply(lambda x: ' '.join(x))

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_sklearn = tfidf_vectorizer.fit_transform(df['article_text_cleaned_string'])
tfidf_feat = tfidf_vectorizer.get_feature_names()

In [18]:
temp = pd.DataFrame(tfidf_sklearn[0].T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["TF-IDF"])
temp = temp.sort_values('TF-IDF', ascending=False)
print (temp.head(25))

                   TF-IDF
nipton           0.300877
vaccine          0.198417
viruses          0.175813
town             0.170453
work             0.167297
humans           0.156807
strain           0.114010
coronavirus      0.113964
remote           0.109308
scientists       0.102198
kxan             0.100292
kron             0.100292
ivanpah          0.100292
vaccines         0.100223
need             0.097373
outbreaks        0.096282
tucked           0.095884
colorful         0.095884
railroad         0.092755
theconversation  0.092755
also             0.090113
million          0.088302
started          0.087954
begun            0.085307
nexstar          0.085218
